In [2]:
import pandas as pd
import numpy as np
from heapq import * 
import math
import random
from scipy.stats import ttest_ind
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.special import comb
from apyori import apriori
pd.options.display.max_seq_items = 1000
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Discrite numerical attributes

In [3]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>20 and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df

### Column transform

In [4]:
def column_transform(df_data,df_label):
    trans = df_data.as_matrix()
    val = df_label.values
    tid = {}
    for i,j in enumerate(trans):
        for m in j:
            tid.setdefault(m, set()).add(i)
    return val,tid

## Impact rule

In [10]:
(3 ** 2) * comb(8,2)

252.0

In [5]:
def opus_ir_pro(k,val,tid,num_col,measurement,adjust=0,pro=0,independent_pro=0):
    k = 100
    me_1 = np.mean(val)
    rule = [(me_1,'empty') for i in range(k)]
    heapify(rule)
    current = set()
    pro_rule = pro_a(k,me_1,rule,val,tid,num_col,current,tid.keys(),measurement,pro,adjust)
    if independent_pro:
        r = [i for i in pro_rule if len(i)>2]
        d = set()
        b = []
        for i,j in enumerate(r):
            for p,q in enumerate(r[:i]):
                s1 = set(r[i][1:-1])
                s2 = set(r[p][1:-1])
                if s2 > s1:
                    temp = [k for k in r[i][-1] if k not in r[p][-1]]
                    if me_1 >= np.mean(temp):
                        d.add(i)
                        b.append(q)
                    else:
                        t , p_val = ttest_ind(val, temp, equal_var=True)
                        if p_val > 0.05:
                            d.add(i)
                            b.append(q)            
                if s1 > s2:
                    temp = [k for k in r[p][-1] if k not in r[i][-1]]
                    if me_1 >= np.mean(temp):
                        d.add(p)
                        b.append(j)
                    else:
                        t , p_val = ttest_ind(val, temp, equal_var=True)
                        if p_val > 0.05:
                            d.add(p)
                            b.append(j)  
        d = list(d)
        d.sort()
        f = []
        for i in d[::-1]:
            f.append(r.pop(i))
        r_test = [i[:-1] for i in pro_rule if len(i)>2], [i[:-1] for i in r],b
        return r_test
    return [i[:-1] for i in pro_rule if len(i)>2]
def pro_a(k,me_1,rule,val,tid,num_col,current,available,measurement,pro=0,adjust=0):
    global counter
    sofar = set()
    for p in available:
        new  = current.copy()
        new.add(p)
        if current == set():
            upper_bound = (max([val[i] for i in tid[p]]))
            val1 = [val[i] for i in tid[p]]
            len_tid = 1
            if measurement == 'mean':
                ir = (np.mean(val1))
            if measurement == 'impact':
                ir = (np.mean(val1) - me_1) * len(tid[p]) 
        else:        
            inter = set.intersection(*map(set,[tid[i] for i in new]))
            upper_bound = (max([val[i] for i in inter]+[0]))
            len_tid = len(new)
            val1 = [val[i] for i in inter]
            if measurement == 'mean':
                ir = (np.mean(val1))
            if measurement == 'impact':
                ir = (np.mean(val1) - me_1) * len(val1) 
        min_heap = heappop(rule)
        if pro:
            if upper_bound > min_heap[0] and not math.isnan(upper_bound):
                if ir > min_heap[0]:
                    parent = [i for i in rule if i[1:-1] and set(i[1:-1])<new]
                    for i in parent:
                        if i[0] >= ir or len(val1) == 1:
                            heappush(rule,min_heap)
                            break
                        t , p_val = ttest_ind(i[-1], val1, equal_var=True)
                        if not adjust:
                            sig = 0.05
                        else:
                            sig = 0.05/((2 ** len_tid) * (3 ** len_tid) * comb(num_col,len_tid))
                        if p_val > sig:
                            heappush(rule,min_heap)
                            break
                    if len(rule) == k-1:
                        heappush(rule,tuple([ir]+list(new)+[val1]))     
                else:
                    heappush(rule,min_heap)
                pro_a(k,me_1,rule,val,tid,num_col,new,sofar,measurement,pro,adjust)
                sofar.add(p)
            else:
                heappush(rule,min_heap)
        else:
            heappush(rule,min_heap)
            if upper_bound > min_heap[0] and not math.isnan(upper_bound):                
                heappushpop(rule, tuple([ir]+list(new)+[val1]))
                pro_a(k,me_1,rule,val,tid,num_col,new,sofar,measurement,pro,adjust)
                sofar.add(p)
    return rule

## Experiment result

## Abalone
This data set is originally owned by Marine Resources Division. The previous usage of this data set is to predict age of abalone. In this experiment, we are going to discover the interrelationship between shucked weight and other attributes. Shuched weight is quantitative attribute and measured by grams. This dataset is sparse dataset as except qualitative attribute Sex, other attributes are quantitative attributes in 4177 records.

In [6]:
df_abalone = pd.read_csv('abalone.data',header = None)
df_abalone.columns = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weigh','Rings']
df_abalone_data = df_abalone[['Sex','Length','Diameter','Height','Whole weight','Viscera weight','Shell weigh','Rings']]
df_abalone_label = df_abalone['Shucked weight']
df_abalone_data = cut_column(df_abalone_data,3)
val,tid = column_transform(df_abalone_data,df_abalone_label)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [141]:
df_abalone = pd.read_csv('abalone.data',header = None)
df_abalone.columns = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weigh','Rings']
df_abalone_data = df_abalone[['Sex','Length','Diameter','Height','Whole weight','Viscera weight','Shell weigh','Rings']]
df_abalone_label = df_abalone['Shucked weight']
df_abalone_data = cut_column(df_abalone_data,3)
val,tid = column_transform(df_abalone_data,df_abalone_label)
mu = opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi,mu_lf = opus_ir_pro(100,val,tid,8,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,8,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,8,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi,impact_lf = opus_ir_pro(100,val,tid,8,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 24
number of records: 4177
100
64
50
25
21
100
12
12
12
12


### Time

#### Time with opus impact rule mining algorithm

In [6]:
df_abalone = pd.read_csv('abalone.data',header = None)
df_abalone.columns = ['Sex','Length','Diameter','Height','Whole weight','Shucked weight','Viscera weight','Shell weigh','Rings']
df_abalone_data = df_abalone[['Sex','Length','Diameter','Height','Whole weight','Viscera weight','Shell weigh','Rings']]
df_abalone_label = df_abalone['Shucked weight']
df_abalone_data = cut_column(df_abalone_data,3)
df_abalone_data.to_csv('df_abalone_data_weka.csv',index = False)
val,tid = column_transform(df_abalone_data,df_abalone_label)
%timeit (opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 loop, best of 3: 1.17 s per loop


#### Time with apriori

In [7]:
transactions = df_abalone_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

834
1 loop, best of 3: 3.45 s per loop


### Non redundancy

In [97]:
[i for i in mu if i[0] == 0.66365340909090909]

[(0.66365340909090909,
  'Rings (11.0, 29.0]',
  'Whole weight (1.03, 2.826]',
  'Height (0.16, 1.13]',
  'Sex M',
  'Diameter (0.465, 0.65]'),
 (0.66365340909090909,
  'Sex M',
  'Whole weight (1.03, 2.826]',
  'Height (0.16, 1.13]',
  'Diameter (0.465, 0.65]',
  'Shell weigh (0.294, 1.005]',
  'Rings (11.0, 29.0]')]

Summary:<br>
In this experiment, we aim to find top 100 impact rules with measurement mean, when rank the target mean of all combination of itemset, 0.66365340909090909 is rank in 47, so these two rules should in the resulting rules.However, in these two rules, we discovered that the target mean of these two rules are the same, and the first rule is the parent rule of the second rule. Thus, with property of non redundancy algorithm, the second rule will be filtered.Beside the theoretical analysis of this problem, we could also use domain expertise to explain why this impact rule is not interesting.First, the only difference of these two impact rules is 'Shell weight (0.294, 1.005)', but in this experiment the target variable is shucked weight, so the second rule is redundant to the first rule by adding meaningless item. What is more, in the consideration of cost, we prefer do less measurements when the target of two rules are the same.

## Forest fire
This data set is created by Cortez and Morais (2017), the originally target of this data set is to predict forest fire area. We will try to find interrelationship between forest fire area and other attributes. Totally 9 attributes are included in this dataset, with 2 qualitative attributes and 7 quantitative attributes in 517 observations. 

In [142]:
def cut_column(df,num):
    for i in df.columns:
        if i == 'rain':
            df.rain[df['rain'] > 0] = 1
        elif df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
        
    return df
df_fire = pd.read_csv('forestfires.csv',header = 0)
df_fire_data = df_fire[[u'X', u'Y', u'temp',u'RH', u'wind', u'rain']]
df_fire_label = np.log(np.log(df_fire[df_fire.columns[-1]]+1)+1)
cut_column(df_fire_data,3)
val,tid = column_transform(df_fire_data,df_fire_label)
mu = opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi,mu_lf = opus_ir_pro(100,val,tid,6,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,6,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,6,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi,impact_lf = opus_ir_pro(100,val,tid,6,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anacon

number of transactions: 17
number of records: 517
100
15
15
14
14
100
12
12
12
12


### Time

#### Time with opus impact rule mining algorithm

In [29]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df
df_fire = pd.read_csv('forestfires.csv',header = 0)
df_fire_data = df_fire[[u'temp',u'RH', u'wind', u'rain']]
df_fire_label = np.log(np.log(df_fire[df_fire.columns[-1]]+1)+1)
cut_column(df_fire_data,3)
df_fire_data.to_csv('df_fire_data_weka.csv',index = False)
val,tid = column_transform(df_fire_data,df_fire_label)
%timeit (opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100 loops, best of 3: 12.5 ms per loop


#### Time with apriori

In [34]:
transactions = df_fire_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

43
100 loops, best of 3: 5.12 ms per loop


### Productivity

In [99]:
[i for i in mu if i[0] in [1.1195413345979819, 1.2010577376605767] and i[1] == 'rain 0.0']

[(1.1195413345979819,
  'rain 0.0',
  'X (0.999, 3.0]',
  'temp (21.4, 33.3]',
  'RH (14.999, 35.0]',
  'Y (4.0, 5.0]'),
 (1.2010577376605767,
  'rain 0.0',
  'temp (21.4, 33.3]',
  'wind (0.399, 3.1]',
  'Y (4.0, 5.0]',
  'RH (14.999, 35.0]',
  'X (0.999, 3.0]')]

Summary:<br>
Target mean value 1.2010577376605767 is higher than 1.1195413345979819, but when applying t test, we found that the p value = 0.79422287525945057, with significant level set to 0.05.
According to the statistical theory, we accept the null hypothesis, there are no significant difference between the mean value of these two samples. Thus, the second rule will not be in the resulting rule as it is not productive.Beside the statistical analysis, we could also explain why this impact rule is not interesting with domain expertise. We discover that the difference between these two rules is item 'wind (0.399, 3.1). As we apply three bin discretization for numerical attributes, the three bins for attribute wind are 'wind (0.399, 3.1)', 'wind (3.1, 4.9)', 'wind (4.9, 9.4)'. while higher wind speed conduct more area of forest fire, add item wind (0.399, 3.1) which is smallest wind speed range does not make too much sense that will conduct more area of forest fire, the rule is discovered by chance and not interesting. 

## Car imports

In [9]:
imports = pd.read_csv('imports-85.data',header = -1)
imports.columns = ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors',
                   'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                   'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                   'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
imports = imports[imports.price.apply(lambda x: x.isdigit())]
df_imports_data = imports[['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors',
                        'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                        'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                        'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg']]
df_imports_label = imports[imports.columns[-1]]
df_imports_label = pd.to_numeric(df_imports_label)
df_imports_data = cut_column(df_imports_data,3)
val,tid = column_transform(df_imports_data,df_imports_label)
counter = set()
mu = opus_ir_pro(100,val,tid,25,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,25,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi, mu_lcvf = opus_ir_pro(100,val,tid,25,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,25,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,25,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi, impact_lcvf = opus_ir_pro(100,val,tid,25,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 303
number of records: 201
100
100
100
100
100
100
100
100
100
100


### Time

#### Time with opus impact rule mining algorithm

In [21]:
imports = pd.read_csv('imports-85.data',header = -1)
imports.columns = ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors',
                   'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                   'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                   'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']
imports = imports[imports.price.apply(lambda x: x.isdigit())]
df_imports_data = imports[['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors',
                        'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                        'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                        'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg']]
df_imports_label = imports[imports.columns[-1]]
df_imports_label = pd.to_numeric(df_imports_label)
df_imports_data = cut_column(df_imports_data,3)
df_imports_data.to_csv('df_imports_data_weka.csv',index = False)
val,tid = column_transform(df_imports_data,df_imports_label)
%timeit (opus_ir_pro(100,val,tid,13,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10 loops, best of 3: 49.3 ms per loop


#### Time with apriori

In [ ]:
transactions = df_imports_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.1, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.1, min_confidence = 0.9))))

Summary:<br>
With impact rule mining algorithm, the run time is less than one seconds but with apriori need excessive computing time. 

## Bank
This database is created by Paulo Cortez and Sérgio Moro (2012). The originally use of this database is to use data mining for bank marketing, and the classification target is to predict if the client will subscribe a term deposit. In our experiment, we will discover interraltionship between quantitative attributes balance and other attributes. Totally 10 attributes are included in this dataset, with 2 quanntitative attributes and 8 qualitative attributes in 45211 observations.

In [150]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df
bank = pd.read_csv('bank.csv',header = 0)
bank_data = bank[[u'age', u'job', u'marital', u'education', u'default',
                  u'housing', u'loan', u'poutcome', u'y']]
bank_label = bank['balance'] 
bank_data = cut_column(bank_data,3)
val,tid = column_transform(bank_data,bank_label)
mu = opus_ir_pro(100,val,tid,9,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,9,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi,mu_lf = opus_ir_pro(100,val,tid,9,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,9,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,9,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi,impact_lf = opus_ir_pro(100,val,tid,9,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 34
number of records: 45211
100
100
100
46
43
100
33
33
29
29


### Time

#### Time with opus impact rule mining algorithm

In [145]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df
bank = pd.read_csv('bank.csv',header = 0)
bank_data = bank[[u'age', u'job', u'marital', u'education', u'default',
                  u'housing', u'loan', u'poutcome', u'y']]
bank_label = bank['balance'] 
bank_data = cut_column(bank_data,3)
bank_data.to_csv('bank_data_weka.csv',index = False)
val,tid = column_transform(bank_data,bank_label)
%timeit (opus_ir_pro(100,val,tid,9,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 loop, best of 3: 3min 46s per loop


#### Time with apriori

In [ ]:
transactions = df_imports_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

Summary:<br>
With impact rule mining algorithm, the run time is 226 seconds but with apriori need excessive computing time. 

## CPU performance

In [143]:
cpu = pd.read_csv('cpu_performance_c.data',header = -1)
cpu.columns = [['vendor','model','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX',
               'RPR','ERP']]
cpu_data = cpu[['MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX']]
cpu_label = cpu[cpu.columns[-2]]
cpu_data = cut_column(cpu_data,3)
val,tid = column_transform(cpu_data,cpu_label)
mu = opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi, mu_lcvf = opus_ir_pro(100,val,tid,6,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,6,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,6,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi, impact_lcvf = opus_ir_pro(100,val,tid,6,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 18
number of records: 209
100
13
7
6
6
100
8
4
6
6


### Time

#### Time with opus impact rule mining algorithm

In [85]:
cpu = pd.read_csv('cpu_performance_c.data',header = -1)
cpu.columns = [['vendor','model','MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX',
               'RPR','ERP']]
cpu_data = cpu[['MYCT','MMIN','MMAX','CACH','CHMIN','CHMAX']]
cpu_label = cpu[cpu.columns[-2]]
cpu_data = cut_column(cpu_data,3)
cpu_data.to_csv('cpu_data_weka.csv',index = False)
val,tid = column_transform(cpu_data,cpu_label)
%timeit (opus_ir_pro(100,val,tid,6,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10 loops, best of 3: 19.9 ms per loop


#### Time with apriori

In [87]:
transactions = cpu_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

60
10 loops, best of 3: 37.8 ms per loop


## Concrete
This dataset is created by Prof. I-Cheng Yeh. There are 9 relative attribute in the data set, and the originally use of this data set is to predict the concrete compressive strength. In our find, we will try to find the interationship between attribute concrete compressive strength and other attributes. The dataset contains 8 quantitative attributes and 1 qualitative attribute with 1030 instances.

In [125]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df

In [144]:
concrete = pd.read_excel('Concrete_Data_c.xls',header = -1)
concrete.columns = [u'Cement', u'Blast Furnace Slag', u'Fly Ash',u'Water',u'Superplasticizer',
                    u'Coarse Aggregate',u'Fine Aggregate', u'Age',u'Concrete compressive strength']
concrete_data = concrete[[u'Cement', u'Blast Furnace Slag', u'Fly Ash',u'Water',u'Superplasticizer',
                          u'Coarse Aggregate',u'Fine Aggregate', u'Age']]
concrete_label = concrete[concrete.columns[-1]]
concrete_data = cut_column(concrete_data,3)
val,tid = column_transform(concrete_data,concrete_label)
mu = opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi, mu_lcvf = opus_ir_pro(100,val,tid,8,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,8,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,8,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi, impact_lcvf = opus_ir_pro(100,val,tid,8,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 20
number of records: 1030
100
64
52
24
19
100
14
8
12
9


### Time

#### Time with opus impact rule mining algorithm

In [310]:
concrete = pd.read_excel('Concrete_Data_c.xls',header = -1)
concrete.columns = [u'Cement', u'Blast Furnace Slag', u'Fly Ash',u'Water',u'Superplasticizer',
                    u'Coarse Aggregate',u'Fine Aggregate', u'Age',u'Concrete compressive strength']
concrete_data = concrete[[u'Cement', u'Blast Furnace Slag', u'Fly Ash',u'Water',u'Superplasticizer',
                          u'Coarse Aggregate',u'Fine Aggregate', u'Age']]
concrete_label = concrete[concrete.columns[-1]]
concrete_data = cut_column(concrete_data,3)
concrete_data.to_csv('concrete_data_weka.csv',index = False)
val,tid = column_transform(concrete_data,concrete_label)
%timeit (opus_ir_pro(100,val,tid,8,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 loop, best of 3: 264 ms per loop


#### Time with apriori

In [309]:
transactions = concrete_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

302
1 loop, best of 3: 291 ms per loop


### Adjust significant test

#### Target value of intersect set and parent set

In [127]:
new =  ['Fine Aggregate (593.999, 754.3]', 'Blast Furnace Slag (-0.001, 116.0]']
inter = set.intersection(*map(set,[tid[i] for i in new]))
val1 = [val[i] for i in inter]
val2 = [val[i] for i in tid['Fine Aggregate (593.999, 754.3]']]
val3 = [val[i] for i in tid['Blast Furnace Slag (-0.001, 116.0]']]

In [128]:
np.mean(val1)

41.61326969512568

In [129]:
np.mean(val2)

38.524487540954631

In [130]:
np.mean(val3)

34.863940637021763

#### T test between intersect and parent rule 'Blast Furnace Slag (-0.001, 116.0]'

In [131]:
t , p_val = ttest_ind(val3, val1, equal_var=True)
p_val

3.6655567996842004e-07

#### T test between intersect and parent rule 'Fine Aggregate (593.999, 754.3]'

In [132]:
t , p_val = ttest_ind(val2, val1, equal_var=True)
p_val

0.027085168958895688

Summary:<br>
For rule 'Fine Aggregate (593.999, 754.3)', 'Blast Furnace Slag (-0.001, 116.0), this rule is non redundant as the target mean of this rule is 41.61326969512568 while the figure of parent rule 'Fine Aggregate (593.999, 754.3)' and , 'Blast Furnace Slag (-0.001, 116.0) are 38.524487540954631 and 34.863940637021763. Meanwhile, this rule is productive when we set significant level to 0.05 and the p-value of t test against the two parent rules are 3.6655567996842004e-07 and 0.027085168958895688. By applying adjust significant level, we are able to ensure that the false positive rate in current level will not be higher than 0.05. Thus, in this problem, as the p-value between target value of 'Fine Aggregate (593.999, 754.3)', 'Blast Furnace Slag (-0.001, 116.0) and 'Fine Aggregate (593.999, 754.3)' is higher than adjust significant level, this rule is not productive.<br>
Beside the statistical analyse, we can also explain why this rule is not interesting by using domain expertise. For concrete, when Blast Furnace Slag is low, the strength of concrete is low. While 'Blast Furnace Slag (-0.001, 116.0) is the lowest range of Blast Furnace Slag, add this item to 'Fine Aggregate (593.999, 754.3)' does not make sense to improve the strength.

## Movie
This dataset is collected from movies data sets on Web, Twitter, YouTube and IMDB. The data set contains 217 movies with 12 relative attributes between 2014 and 2015. In our experiment, we are going to discover the interrelationship between profit margin and 
other attributes.

In [145]:
def cut_column(df,num):
    for i in df.columns:
        if df[i].nunique()>num and  df[i].dtype != 'O':
            df[i] = pd.qcut(df[i].values, num, retbins=True,duplicates= 'drop')[0]
        df[i] = str(i) + ' ' + df[i].astype(str)
    return df

In [146]:
movie = pd.read_excel('movie_c.xlsx',header = -1)
movie = movie[-movie['Budget'].isnull()]
movie = movie.drop(movie.index[[-1]])
movie_data = movie[[u'Year', u'Ratings', u'Genre',
                    u'Screens', u'Sequel', u'Sentiment', u'Views', u'Likes', u'Dislikes',
                    u'Comments', u'Aggregate Followers']]
movie_label = (movie[u'Gross'] - movie[u'Budget'])/movie[u'Budget']
movie_data = cut_column(movie_data,3)
val,tid = column_transform(movie_data,movie_label)
mu = opus_ir_pro(100,val,tid,11,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,11,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi, mu_lcvf = opus_ir_pro(100,val,tid,11,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,11,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,11,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi, impact_lcvf = opus_ir_pro(100,val,tid,11,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi]
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 32
number of records: 229
100
100
100
72
72
100
82
65
52
52


### Time

#### Time with opus impact rule mining algorithm

In [244]:
movie = pd.read_excel('movie_c.xlsx',header = -1)
movie = movie[-movie['Budget'].isnull()]
movie = movie.drop(movie.index[[-1]])
movie_data = movie[[u'Year', u'Ratings', u'Genre',
                    u'Screens', u'Sequel', u'Sentiment', u'Views', u'Likes', u'Dislikes',
                    u'Comments', u'Aggregate Followers']]
movie_label = (movie[u'Gross'] - movie[u'Budget'])/movie[u'Budget']
movie_data = cut_column(movie_data,3)
movie_data.to_csv('movie_data_weka.csv',index = False)
val,tid = column_transform(movie_data,movie_label)
%timeit (opus_ir_pro(100,val,tid,12,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10 loops, best of 3: 112 ms per loop


#### Time with apriori

In [306]:
transactions = movie_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

2877
1 loop, best of 3: 13.4 s per loop


### Independent productivity

In [135]:
set([i[:-1] for i in impact_af if 'Year 2015' in i and  ('Dislikes (-0.001, 155.0]' in i or  'Views (697.999, 1142295.0]' in i)])

{(19.826017951777104,
  'Year 2015',
  'Dislikes (-0.001, 155.0]',
  'Ratings (6.8, 8.7]'),
 (20.182948966994733,
  'Aggregate Followers (276457.667, 2376333.333]',
  'Year 2015',
  'Sentiment (0.0, 3.0]',
  'Dislikes (-0.001, 155.0]'),
 (21.385755984538594,
  'Views (697.999, 1142295.0]',
  'Year 2015',
  'Sentiment (0.0, 3.0]',
  'Ratings (6.8, 8.7]')}

In [136]:
new =  ['Year 2015']
inter = set.intersection(*map(set,[tid[i] for i in new]))
val1 = [val[i] for i in inter]
np.mean(val1)

1.2567366408017537

In [137]:
new =  ['Year 2015',
  'Dislikes (-0.001, 155.0]',
  'Ratings (6.8, 8.7]']
inter = set.intersection(*map(set,[tid[i] for i in new]))
val2 = [val[i] for i in inter]
np.mean([i for i in val1 if i not in val2])

0.94526441306376707

In [219]:
np.mean(val)

1.1537886743973702

Summary:<br>
Independent productivity is important to filter uninteresting parent impact rules. We will examine independent productivity for productive rules. In the experiment against dataset movie, we discovered 88 productive rules with configuration ‘OPUS_impact_pro’-search by impact and perform t test. One of the resulting rule is: (6.8975137490936964, 'Year 2015'). Before we examine independent productivity, we would verify this rule could fulfil the measurement of non-redundancy and productivity.  The target impact value of impact rule 'Year 2015' is in the top 100 list. At the mean time, as this rule formed by one element, so there are no parent rule. Thus, this rule is productive and non-redundant. However, the fact hidden in this pattern is not meaningful. Independent productivity algorithm proves this hypothesis by filter this rule. In the resulting productive rules, we found 3 supersets for this itemset. Take one of the superset 'Year 2015', 'Dislikes (-0.001, 155.0)', 'Ratings (6.8, 8.7)' as example, when we exclude the records with itemsets 'Year 2015', 'Dislikes (-0.001, 155.0)', 'Ratings (6.8, 8.7)' for the itemset 'Year 2015', 5 records are removed but the mean target value dropped from 1.2567366408017537 to 0.94526441306376707. While the global mean is 1.1537886743973702, the impact turned to negative. Thus rule 'Year 2015' is not independent productive.<br>
We could explain why this impact rule is uninteresting by using domain expertise beside the statistical analysis. ‘A movie produced in 2015’ does not make too much sense that it can earn more money. However, we have more knowledge about the film by appending more items to the rule, such as ‘the ratings of this movie is high’ and ‘the dislike of this movie is low’. With principle of independent productivity, we are able to discovery rules with sufficient knowledge.


## Energy
The dataset is originally created by Angeliki Xifara and processed by Athanasios Tsanas. The past use of this dataset is to implement energy efficiency analysis with 12 different building and to predict the attribute heating load. In our experiment, we are going to find interrelationship between attribute heating load and other attributes, There are 8 features in the dataset, 4 qualitative and 4 quantitative attributes with 768 instance.

In [147]:
energy = pd.read_excel('ENB2012_data_c.xlsx',header = -1)
energy.columns = ['Relative Compactness','Surface Area','Wall Area','Roof Area','Overall Height',
                  'Orientation','Glazing Area','Glazing Area Distribution','Heating Load','Cooling Load']
energy_data = energy[['Relative Compactness','Surface Area','Wall Area','Roof Area','Overall Height',
                      'Orientation','Glazing Area','Glazing Area Distribution']]
energy_label = energy['Heating Load']
energy_data = cut_column(energy_data,3)
val,tid = column_transform(energy_data,energy_label)
mu = opus_ir_pro(100,val,tid,12,'mean',adjust=0,pro=0,independent_pro=0)
mu_a, mu_ai,mu_af = opus_ir_pro(100,val,tid,12,'mean',adjust=0,pro=1,independent_pro=1)
mu_lcv, mu_lcvi, mu_lcvf = opus_ir_pro(100,val,tid,12,'mean',adjust=1,pro=1,independent_pro=1)
impact = opus_ir_pro(100,val,tid,12,'impact',adjust=0,pro=0,independent_pro=0)
impact_a, impact_ai,impact_af = opus_ir_pro(100,val,tid,12,'impact',adjust=0,pro=1,independent_pro=1)
impact_lcv, impact_lcvi, impact_lcvf = opus_ir_pro(100,val,tid,12,'impact',adjust=1,pro=1,independent_pro=1)
l = [mu,mu_a,mu_ai,mu_lcv,mu_lcvi,impact,impact_a,impact_ai,impact_lcv,impact_lcvi] 
print ('number of transactions' + ': ' + str(len(tid.keys())))
print ('number of records' + ': ' + str(len(val)))
for i in l:
    print len(i)

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


number of transactions: 22
number of records: 768
100
34
29
33
29
100
14
11
14
11


### Time

#### Time with opus impact rule mining algorithm

In [248]:
energy = pd.read_excel('ENB2012_data_c.xlsx',header = -1)
energy.columns = ['Relative Compactness','Surface Area','Wall Area','Roof Area','Overall Height',
                  'Orientation','Glazing Area','Glazing Area Distribution','Heating Load','Cooling Load']
energy_data = energy[['Relative Compactness','Surface Area','Wall Area','Roof Area','Overall Height',
                      'Orientation','Glazing Area','Glazing Area Distribution']]
energy_label = energy['Heating Load']
energy_data = cut_column(energy_data,3)
energy_data.to_csv('energy_data_weka.csv',index = False)
val,tid = column_transform(energy_data,energy_label)
%timeit (opus_ir_pro(100,val,tid,12,'mean',adjust=0,pro=0,independent_pro=0))

/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jason/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 loop, best of 3: 191 ms per loop


#### Time with apriori

In [307]:
transactions = energy_data.values.tolist()
print (len(list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))
%timeit ((list(apriori(transactions,min_support = 0.05, min_confidence = 0.9))))

861
1 loop, best of 3: 421 ms per loop
